In [1]:
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal
import warnings

warnings.filterwarnings("ignore")

root_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(root_path)

In [ ]:
from core.data_sources import CLOBDataSource

clob = CLOBDataSource()

candles = await clob.get_candles_last_days(connector_name="binance_perpetual", trading_pair="ADA-USDT", interval="1h", days=30)

2025-01-26 22:01:03,863 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1755f9690>
2025-01-26 22:01:03,865 - asyncio - ERROR - Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x1122536a0>, 397075.118181)])']
connector: <aiohttp.connector.TCPConnector object at 0x1755f96c0>


In [9]:
fig = candles.fig(type="candles")
fig.update_layout(paper_bgcolor="white", plot_bgcolor="white")
fig.update_layout(font=dict(color="black"))
fig.show()

In [7]:
%reload_ext autoreload

from features.moving import MovingConfig, Moving
from features.volmom import VolMomConfig, VolMom
from features.trend_str import TrendStrConfig, TrendStr
from features.vol_dynamic_feat import VolDynConfig, VolDyn

In [19]:
config = MovingConfig(ema_short=10, sma_long=50)
candles.add_feature(Moving(config))

volmom_config = VolMomConfig(roc_length=14, bbands_length=20, bbands_std=2)
candles.add_feature(VolMom(volmom_config))

trend_config = TrendStrConfig(rsi_length=14, adx_period=14)
candles.add_feature(TrendStr(trend_config))

voldyn_config = VolDynConfig(spike_zscore=2)
candles.add_feature(VolDyn(voldyn_config))

candles.data.tail()

,timestamp,open,high,low,close,volume,quote_asset_volume,n_trades,taker_buy_base_volume,taker_buy_quote_volume,...,bb_upper,bb_middle,bb_lower,rsi,adx,+di,-di,obv,vol_zscore,vol_spike
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-01-22 04:00:00,1737518400,1.0027,1.0065,0.9939,1,13416976,13409843.5656,33698,6079790,6079583.5355,...,1.02633467,1.002365,0.97839533,47.05986569,19.46271731,17.59621227,21.48327031,-1034449745,-0.72727042,False
2025-01-22 05:00:00,1737522000,1,1.0037,0.9971,0.9974,9816865,9822822.7911,27672,4751435,4755344.3766,...,1.02289398,1.003715,0.98453602,45.91538888,18.78299067,17.1002182,20.8777096,-1044266610,-0.86377498,False
2025-01-22 06:00:00,1737525600,0.9975,1.0013,0.9929,0.9997,11631158,11596902.6996,32507,5462932,5447231.9038,...,1.02242383,1.004205,0.98598617,47.14006737,18.46316073,16.46413173,21.96098788,-1032635452,-0.79498286,False
2025-01-22 07:00:00,1737529200,0.9997,0.9998,0.9869,0.9902,37507021,37261744.8929,49760,15586590,15488085.3996,...,1.02286531,1.003725,0.98458469,42.8264564,18.59039366,15.50997295,23.38379045,-1070142473,0.18614618,False
2025-01-22 08:00:00,1737532800,0.9901,0.9922,0.9874,0.9875,1656415,1640032.2257,4717,781901,774357.6822,...,1.0234687,1.003115,0.9827613,41.65966978,18.70853853,15.15795988,22.85307387,-1071798888,-1.17319285,False


In [62]:
import plotly.graph_objects as go

fig.add_trace(
    go.Scatter(x=candles.data.index, y=candles.data["ema_short"], mode="lines", name="EMA Short", line=dict(color="blue"))
)

fig.add_trace(go.Scatter(x=candles.data.index, y=candles.data["sma_long"], mode="lines", name="SMA Long", line=dict(color="red")))

fig.show()

In [21]:
candles.data.count()

timestamp                 720
open                      720
high                      720
low                       720
close                     720
volume                    720
quote_asset_volume        720
n_trades                  720
taker_buy_base_volume     720
taker_buy_quote_volume    720
ema_short                 711
sma_long                  671
roc                       706
bb_upper                  701
bb_middle                 701
bb_lower                  701
rsi                       706
adx                       693
+di                       706
-di                       706
obv                       720
vol_zscore                720
vol_spike                 720
dtype: int64

In [51]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

candles_df = candles.data

fig = make_subplots(rows=6, cols=1, shared_xaxes=True, vertical_spacing=0.02, row_heights=[0.5, 0.1, 0.1, 0.1, 0.1, 0.1])

# Add candlestick
fig.add_trace(
    go.Candlestick(
        x=candles_df.index,
        open=candles_df["open"],
        high=candles_df["high"],
        low=candles_df["low"],
        close=candles_df["close"],
        name="Candlesticks",
        hoverlabel=dict(bgcolor="white", font=dict(color="black")),  # Customize hover label
    ),
    row=1,
    col=1,
)

# Add Bollinger Bands
bb_upper = "bb_upper"
bb_lower = "bb_lower"
fig.add_trace(
    go.Scatter(
        x=candles_df.index,
        y=candles_df[bb_upper],
        line=dict(color="rgba(173, 204, 255, 0.7)"),
        name="BB Upper",
        hovertemplate="BB Upper: %{y}<extra></extra>",  # Custom tooltip
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=candles_df.index,
        y=candles_df[bb_lower],
        line=dict(color="rgba(173, 204, 255, 0.7)"),
        fill="tonexty",
        fillcolor="rgba(173, 204, 255, 0.1)",
        name="BB Lower",
        hovertemplate="BB Lower: %{y}<extra></extra>",  # Custom tooltip
    ),
    row=1,
    col=1,
)

# Add EMA and SMA
fig.add_trace(
    go.Scatter(
        x=candles_df.index,
        y=candles_df["ema_short"],
        mode="lines",
        name="EMA Short",
        line=dict(color="blue"),
        hovertemplate="EMA Short: %{y}<extra></extra>",  # Custom tooltip
    )
)
fig.add_trace(
    go.Scatter(
        x=candles_df.index,
        y=candles_df["sma_long"],
        mode="lines",
        name="SMA Long",
        line=dict(color="red"),
        hovertemplate="SMA Long: %{y}<extra></extra>",  # Custom tooltip
    )
)

# Add Volume Bars
fig.add_trace(
    go.Bar(
        x=candles_df.index,
        y=candles_df["volume"],
        name="Volume",
        marker_color="blue",
        hovertemplate="Volume: %{y}<extra></extra>",  # Custom tooltip
    ),
    row=2,
    col=1,
)

# Highlight Volume Spikes
fig.add_trace(
    go.Scatter(
        x=candles_df.index[candles_df["vol_spike"]],
        y=candles_df.loc[candles_df["vol_spike"], "volume"],
        mode="markers",
        name="Volume Spike",
        marker=dict(color="red", size=10, symbol="circle"),
        hovertemplate="Volume Spike: %{y}<extra></extra>",  # Custom tooltip
    ),
    row=2,
    col=1,
)

# On-Balance Volume (OBV) Line
fig.add_trace(
    go.Scatter(
        x=candles_df.index,
        y=candles_df["obv"],
        mode="lines",
        name="OBV",
        line=dict(color="orange"),
        hovertemplate="OBV: %{y}<extra></extra>",  # Custom tooltip
    ),
    row=3,
    col=1,
)

# Add RSI (Relative Strength Index)
fig.add_trace(
    go.Scatter(
        x=candles_df.index,
        y=candles_df["rsi"],
        mode="lines",
        name="RSI",
        line=dict(color="purple"),
        hovertemplate="RSI: %{y}<extra></extra>",  # Custom tooltip
    ),
    row=4,
    col=1,
)

# Add ADX, +DI, -DI
fig.add_trace(
    go.Scatter(
        x=candles_df.index,
        y=candles_df["adx"],
        mode="lines",
        name="ADX",
        line=dict(color="green"),
        hovertemplate="ADX: %{y}<extra></extra>",  # Custom tooltip
    ),
    row=5,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=candles_df.index,
        y=candles_df["+di"],
        mode="lines",
        name="+DI",
        line=dict(color="orange"),
        hovertemplate="+DI: %{y}<extra></extra>",  # Custom tooltip
    ),
    row=5,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=candles_df.index,
        y=candles_df["-di"],
        mode="lines",
        name="-DI",
        line=dict(color="red"),
        hovertemplate="-DI: %{y}<extra></extra>",  # Custom tooltip
    ),
    row=5,
    col=1,
)

# Add ROC (Rate of Change)
fig.add_trace(
    go.Scatter(
        x=candles_df.index,
        y=candles_df["roc"],
        mode="lines",
        name="ROC",
        line=dict(color="green"),
        hovertemplate="ROC: %{y}<extra></extra>",  # Custom tooltip
    ),
    row=6,
    col=1,
)

# Update layout for dark theme and full width
fig.update_layout(
    width=None,  # Set width to None to allow full width
    height=1200,
    autosize=True,  # Allow the chart to resize dynamically
    font=dict(color="#e1e1e1"),
    plot_bgcolor="#1e1e1e",
    paper_bgcolor="#1e1e1e",
    xaxis_rangeslider_visible=False,
    legend=dict(bgcolor="rgba(0,0,0,0)"),
    showlegend=False,
    hovermode="x unified",  # Show tooltips for all traces at the same x-value
)

# Update axes
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor="#323232", zeroline=False)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor="#323232", zeroline=False)

# Show the plot
fig.show()

In [68]:
candles_df.columns

Index(['timestamp', 'open', 'high', 'low', 'close', 'volume',
       'quote_asset_volume', 'n_trades', 'taker_buy_base_volume',
       'taker_buy_quote_volume', 'ema_short', 'sma_long', 'roc', 'bb_upper',
       'bb_middle', 'bb_lower', 'rsi', 'adx', '+di', '-di', 'obv',
       'vol_zscore', 'vol_spike', 'signal'],
      dtype='object')

In [34]:
def compute_signal(candles):
    # Price crosses above the EMA: Bullish signal (go long).
    mov_av_long_signal = candles["ema_short"] > candles["sma_long"]

    # Price crosses below the EMA: Bearish signal (go short).
    mov_av_short_signal = candles["ema_short"] < candles["sma_long"]

    # ROC > 0: Uptrend gaining momentum.
    roc_long_signal = candles["roc"] > 0
    # ROC < 0: Downtrend gaining momentum.
    roc_short_signal = candles["roc"] < 0
    # ROC crossing a high threshold (e.g., 5%) signals strong momentum bursts, useful for entering.

    # Overbought or breakout confirmation (short signal)
    bb_long_signal = candles["close"] > candles["bb_upper"]
    # Oversold or trend exhaustion (long signal)
    bb_short_signal = candles["close"] < candles["bb_lower"]

    # # Identify volatility squeezes (Narrow bands)
    # candles['band_width'] = candles['upper_band'] - candles['lower_band']
    # candles['volatility_squeeze'] = (candles['band_width'] / candles['band_width'].rolling(window=20).mean()) < 0.8

    # RSI < 30: Oversold (consider entering long positions).
    rsi_long_signal = candles["rsi"] < 30
    # RSI > 70: Overbought (consider taking profit or going short).
    rsi_short_signal = candles["rsi"] > 70

    # ADX > 25: Strong trend.
    adx_long_signal = (candles_df["adx"] > 25) & (candles_df["+di"] > candles_df["-di"])
    # ADX < 20: Weak or ranging market.
    adx_short_signal = (candles["adx"] < 20) & (candles["-di"] > candles["+di"])
    # Use +DI and -DI for direction confirmation:
    # +DI > -DI: Uptrend.
    # -DI > +DI: Downtrend.

    # Volume Confirmation
    avg_volume = candles["volume"].rolling(window=21).mean()
    strong_volume = candles["volume"] > 2 * avg_volume  # Strong breakout/breakdown volume confirmation
    weak_volume = candles["volume"] < avg_volume  # Low conviction for trend moves

    # On-Balance Volume (OBV)
    obv_change = candles["obv"].diff()
    obv_trend = (obv_change > 0).astype(int) - (obv_change < 0).astype(int)  # +1 for up, -1 for down

    # OBV divergence signals
    price_diff = candles["close"].diff()
    obv_divergence = ((price_diff > 0) & (obv_change <= 0)) | (  # Price up, OBV flat/down
        (price_diff < 0) & (obv_change >= 0)
    )  # Price down, OBV flat/up

    # Whale Trades or Volume Spikes
    whale_trade = candles["vol_zscore"] > 2  # Significant volume spike based on Z-score

    # long_signal = mov_av_long_signal & roc_long_signal & bb_long_signal & rsi_long_signal & adx_long_signal & strong_volume & (obv_trend > 0)
    # short_signal = mov_av_short_signal & roc_short_signal & bb_short_signal & rsi_short_signal & adx_short_signal & weak_volume  & (obv_trend < 0)

    # rsi_long_signal  bb_long_signal
    # rsi_short_signal  bb_short_signal
    long_signal = mov_av_long_signal & roc_long_signal
    short_signal = mov_av_short_signal & roc_short_signal

    candles["signal"] = 0
    candles.loc[long_signal, "signal"] = 1
    candles.loc[short_signal, "signal"] = -1

    # Avoid look forward bias?
    candles["signal"].shift(1)

    return candles

In [35]:
candles_df = candles.data

signal_df = compute_signal(candles_df)
signal_df["signal"].value_counts()

signal
 0    683
-1     23
 1     14
Name: count, dtype: int64

In [36]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

plot_df = signal_df

fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02, subplot_titles=("OHLC", "Signal"), row_heights=[0.6, 0.2]
)

# Add candlestick
fig.add_trace(
    go.Candlestick(
        x=plot_df.index,
        open=plot_df["open"],
        high=plot_df["high"],
        low=plot_df["low"],
        close=plot_df["close"],
        name="Candlesticks",
    ),
    row=1,
    col=1,
)

# Add the signal line
fig.add_trace(
    go.Scatter(x=plot_df.index, y=plot_df["signal"], mode="lines", name="Signal", line=dict(color="white")), row=2, col=1
)


# Update layout for dark theme
fig.update_layout(
    # title=f'{exchange} - {trading_pair} - {timeframe}',
    width=1200,
    height=800,
    font=dict(color="#e1e1e1"),
    plot_bgcolor="#1e1e1e",
    paper_bgcolor="#1e1e1e",
    xaxis_rangeslider_visible=False,
    legend=dict(bgcolor="rgba(0,0,0,0)"),
    yaxis=dict(title="Price"),
    yaxis2=dict(title="Signal", showgrid=False),
    showlegend=False,
)

# Update axes
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor="#323232", zeroline=False)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor="#323232", zeroline=False)

# Show the plot
fig.show()

In [5]:
config = clob.get_connector_config_map("binance_perpetual")
config

{'binance_perpetual_api_key': 'VJNHTUrlzmERV4OyEcRdiVIfZmhlfuGWybeZ14YTZ4BmQkVL2ClWSYIAUKPvPBef',
 'binance_perpetual_api_secret': 'OmdPyWzOp1aoAKRN3w5TKhHJcm3Pog0c6gZrcNfHHFRIyhizKL8bLK3MJwLmxXHR'}

In [63]:
df1 = candles.data.head(10)
df1

,timestamp,open,high,low,close,volume,quote_asset_volume,n_trades,taker_buy_base_volume,taker_buy_quote_volume
timestamp,,,,,,,,,,
2024-12-27 23:00:00,1735340400,0.874,0.8772,0.8695,0.8768,10840940,9473571.3227,36454,5579494,4877080.0765
2024-12-28 00:00:00,1735344000,0.8768,0.8802,0.872,0.875,13961986,12233002.5878,44184,5914110,5185232.8744
2024-12-28 01:00:00,1735347600,0.875,0.8762,0.8656,0.8699,15666694,13636663.0307,40080,7700492,6703486.711
2024-12-28 02:00:00,1735351200,0.87,0.8753,0.8667,0.8671,10724467,9348214.1631,27547,6528535,5690752.038
2024-12-28 03:00:00,1735354800,0.867,0.8758,0.8668,0.8756,8406739,7331804.4842,23068,5166594,4505927.0954
2024-12-28 04:00:00,1735358400,0.8756,0.8758,0.8706,0.8737,6950806,6070859.327,21748,3279308,2864096.3544
2024-12-28 05:00:00,1735362000,0.8737,0.8827,0.8737,0.8786,11537773,10146583.5625,30064,5772225,5076010.2301
2024-12-28 06:00:00,1735365600,0.8787,0.8807,0.8688,0.8702,9012246,7867033.7282,27216,3931054,3432037.2012
2024-12-28 07:00:00,1735369200,0.8702,0.8741,0.8658,0.8676,8729834,7593202.4976,26770,4628836,4026191.4447


In [64]:
df2 = df
df2

,symbol,fundingTime,fundingRate,markPrice
0,ADAUSDT,2024-12-27 00:00:00.000,0.00008162,0.86048051
1,ADAUSDT,2024-12-27 08:00:00.000,0.00003224,0.86348117
2,ADAUSDT,2024-12-27 16:00:00.000,0.0001,0.87410000
3,ADAUSDT,2024-12-28 00:00:00.001,0.0001,0.87680000
4,ADAUSDT,2024-12-28 08:00:00.000,0.00008063,0.86760000
5,ADAUSDT,2024-12-28 16:00:00.000,0.0001,0.87520000
6,ADAUSDT,2024-12-29 00:00:00.000,0.0001,0.88920000


In [67]:
df2

,symbol,fundingTime,fundingRate,markPrice
0,ADAUSDT,2024-12-27 00:00:00.000,0.00008162,0.86048051
1,ADAUSDT,2024-12-27 08:00:00.000,0.00003224,0.86348117
2,ADAUSDT,2024-12-27 16:00:00.000,0.0001,0.87410000
3,ADAUSDT,2024-12-28 00:00:00.001,0.0001,0.87680000
4,ADAUSDT,2024-12-28 08:00:00.000,0.00008063,0.86760000
5,ADAUSDT,2024-12-28 16:00:00.000,0.0001,0.87520000
6,ADAUSDT,2024-12-29 00:00:00.000,0.0001,0.88920000


In [ ]:
import pandas as pd
import numpy as np


def join_dataframes_on_closest_timestamp(df1, df2):
    # Convert timestamp columns to datetime if they are not already
    df1["timestamp"] = pd.to_datetime(df1.index)
    df2["fundingTime"] = pd.to_datetime(df2["fundingTime"])

    # Initialize an empty list to store the closest funding rates
    funding_rates = []

    # Iterate over each row in df1
    for index, row in df1.iterrows():
        # Calculate the time difference between the current timestamp and all fundingTimes
        time_diffs = (df2["fundingTime"] - row["timestamp"]).abs()

        # Find the closest fundingTime that is less than or equal to the current timestamp
        valid_diffs = df2["fundingTime"][df2["fundingTime"] <= row["timestamp"]]
        if not valid_diffs.empty:
            closest_time = valid_diffs.max()  # Get the latest fundingTime <= current timestamp
            closest_index = df2[df2["fundingTime"] == closest_time].index[0]
        else:
            # If no fundingTime is <= current timestamp, use the earliest fundingTime
            closest_index = time_diffs.idxmin()

        # Append the corresponding fundingRate to the list
        funding_rates.append(df2.loc[closest_index, "fundingRate"])

    # Add the fundingRate column to df1
    df1["fundingRate"] = funding_rates

    # Return the resulting DataFrame
    return df1


# Example usage:
# Assuming df1 and df2 are already defined as per your example
df3 = join_dataframes_on_closest_timestamp(df1, df2)
df3

,timestamp,open,high,low,close,volume,quote_asset_volume,n_trades,taker_buy_base_volume,taker_buy_quote_volume,fundingRate
timestamp,,,,,,,,,,,
2024-12-27 23:00:00,2024-12-27 23:00:00,0.874,0.8772,0.8695,0.8768,10840940,9473571.3227,36454,5579494,4877080.0765,0.0001
2024-12-28 00:00:00,2024-12-28 00:00:00,0.8768,0.8802,0.872,0.875,13961986,12233002.5878,44184,5914110,5185232.8744,0.0001
2024-12-28 01:00:00,2024-12-28 01:00:00,0.875,0.8762,0.8656,0.8699,15666694,13636663.0307,40080,7700492,6703486.711,0.0001
2024-12-28 02:00:00,2024-12-28 02:00:00,0.87,0.8753,0.8667,0.8671,10724467,9348214.1631,27547,6528535,5690752.038,0.0001
2024-12-28 03:00:00,2024-12-28 03:00:00,0.867,0.8758,0.8668,0.8756,8406739,7331804.4842,23068,5166594,4505927.0954,0.0001
2024-12-28 04:00:00,2024-12-28 04:00:00,0.8756,0.8758,0.8706,0.8737,6950806,6070859.327,21748,3279308,2864096.3544,0.0001
2024-12-28 05:00:00,2024-12-28 05:00:00,0.8737,0.8827,0.8737,0.8786,11537773,10146583.5625,30064,5772225,5076010.2301,0.0001
2024-12-28 06:00:00,2024-12-28 06:00:00,0.8787,0.8807,0.8688,0.8702,9012246,7867033.7282,27216,3931054,3432037.2012,0.0001
2024-12-28 07:00:00,2024-12-28 07:00:00,0.8702,0.8741,0.8658,0.8676,8729834,7593202.4976,26770,4628836,4026191.4447,0.0001
